# Convolutional Neural Network

### Importing the libraries

In [16]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.src.legacy.preprocessing.image import ImageDataGenerator

In [9]:
tf.__version__
ImageDataGenerator

keras.src.legacy.preprocessing.image.ImageDataGenerator

## Part 1 - Data Preprocessing

### Preprocessing the Training set


This code snippet is part of a process to prepare image data for training a machine learning model using Keras. Here's a breakdown of what each line does:

## Explanation of Code

### 1. **Image Data Augmentation**
```python
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
```
The ImageDataGenerator class in Keras is used to augment image data in real-time and preprocess it. The parameters specified here configure how the data should be augmented.

#### rescale=1./255:

Normalizes the pixel values of the images by scaling them down from the range [0, 255] to [0, 1]. This helps the model train more effectively since normalized data often leads to faster convergence.

#### shear_range=0.2:

Applies random shear transformations to the images. Shearing changes the perspective of an image along a particular axis, making the model more robust to such variations.

#### zoom_range=0.2:

Randomly zooms into the images by up to 20%. This makes the model more robust to scale variations.

#### horizontal_flip=True:

Randomly flips images horizontally. This is useful when horizontal orientation is not a distinguishing factor for the class label (e.g., detecting cats or dogs).

### 2. **training_set = train_datagen.flow_from_directory(...)**

This line generates batches of augmented image data from a directory. Let's break down the parameters:

#### '/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/training_set':

Specifies the directory containing the training images. Inside this directory, images are expected to be organized in subdirectories, where each subdirectory corresponds to a class.

#### target_size=(64, 64):

Resizes all images to 64x64 pixels. This ensures uniformity in input dimensions, which is necessary for training the model.

#### batch_size=32:

Defines the number of images per batch. During training, the model processes images in chunks of 32, rather than one by one, to optimize computational efficiency.

#### class_mode='binary':

Indicates that the classification task is binary (e.g., distinguishing between two classes like "cats" and "dogs"). The labels will be either 0 or 1.

#### What Happens Here?
The ImageDataGenerator applies augmentation and preprocessing to the images in the training set in real-time.
The flow_from_directory method reads images from the specified directory, resizes them, applies the defined augmentations, and generates batches of data for training.

#### Output
This setup returns a Python generator (training_set) that can be used directly with the fit or fit_generator method of a Keras model. The generator ensures efficient memory usage by loading and processing only a batch of images at a time instead of loading the entire dataset into memory.

#### Purpose of Data Augmentation
The goal of augmentation is to artificially increase the diversity of the training dataset by introducing transformations. This helps prevent overfitting, especially when the dataset is small.


In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/training_set', 
                                                 target_size=(64,64), 
                                                 batch_size=32, 
                                                 class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

The `ImageDataGenerator` class is used here to preprocess the test images. The only parameter specified is:

#### rescale=1./255:
Normalizes pixel values from [0, 255] to [0, 1]. This is crucial for maintaining consistency with the preprocessing applied to the training data.

**Note:** Unlike the training data generator, no data augmentation (like shearing or flipping) is applied here. The test set should represent real-world data without modifications, ensuring accurate evaluation.

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('/home/vpsr/Desktop/python/Machine Learning Resources/dataset/dataset/test_set', 
                                            target_size=(64,64), 
                                            batch_size=32, 
                                            class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [17]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-21 09:31:08.146358: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Step 2 - Pooling

In [19]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [20]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [21]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [22]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [23]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

### Training the CNN on the Training set and evaluating it on the Test set

## Part 4 - Making a single prediction